In [ ]:
import os
import site
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/auto-diagnosis/')
%cd /home/schirrmr/
# switch to cpu
os.environ['THEANO_FLAGS'] = 'floatX=float32,device=cpu,nvcc.fastmath=True'

%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'svg' 
matplotlib.rcParams['figure.figsize'] = (12.0, 4.0)
matplotlib.rcParams['font.size'] = 7

import matplotlib.lines as mlines
import seaborn
seaborn.set_style('darkgrid')
import logging
import importlib
importlib.reload(logging) # see https://stackoverflow.com/a/21475297/1469195
log = logging.getLogger()
log.setLevel('DEBUG')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.DEBUG, stream=sys.stdout)

from braindecode.torch_ext.util import np_to_var

In [ ]:
import torch as th

In [ ]:
os.sys.path.insert(0, '/home/schirrmr/code/auto-diagnosis/')

In [ ]:
from autodiag.dataset import load_data, DiagnosisSet, get_all_sorted_file_names_and_labels

In [ ]:
all_file_names, labels = get_all_sorted_file_names_and_labels('train')

In [ ]:
from numpy.random import RandomState
rng = RandomState(384734)
in_var = np_to_var(rng.randn(3,5,18,2), dtype=np.float32)
percentile = th.kthvalue(th.abs(in_var.view(-1)), int(0.95 * in_var.numel()))
cutoff = percentile * 2
# store das die ganze zeit, und beim forward pass clampe die werte dann zu +-cutoff

In [ ]:
lengths = np.load(
            '/home/schirrmr/code/auto-diagnosis/sorted-recording-lengths.npy')
mask = lengths < 35 * 60
cleaned_file_names = np.array(all_file_names)[mask]
cleaned_labels = labels[mask]

In [ ]:
sec_to_cut = 60
preproc_functions = []
duration_recording_mins = 3
preproc_functions.append(
    lambda data, fs: (data[:, int(sec_to_cut * fs):-int(
        sec_to_cut * fs)], fs))
preproc_functions.append(
    lambda data, fs: (data[:, :int(duration_recording_mins * 60 * fs)], fs))

In [ ]:
import torch.nn.functional as F
def compute_min_max_diffs(x, window_len, cuda):
    x_var = np_to_var(x)
    if x_var.dim() == 2:
        x_var = x_var.unsqueeze(0)
    if cuda:
        x_var = x_var.cuda()

    maxs = F.max_pool1d(x_var,window_len, stride=1)
    mins = F.max_pool1d(-x_var,window_len, stride=1)

    diffs = maxs + mins
    return diffs

In [ ]:
train_inds = np.int32(np.arange(10))
valid_inds = np.int32(np.arange(10,12))
test_inds = np.int32(np.arange(12,15))

In [ ]:
import resampy
sec_to_cut = 60
preproc_functions = []
duration_recording_mins = 3
sampling_freq = 100
preproc_functions.append(
    lambda data, fs: (data[:, int(sec_to_cut * fs):-int(
        sec_to_cut * fs)], fs))
preproc_functions.append(
    lambda data, fs: (data[:, :int(duration_recording_mins * 60 * fs)], fs))
preproc_functions.append(lambda data, fs: (resampy.resample(data, fs,
                                                            sampling_freq,
                                                            axis=1,
                                                            filter='kaiser_fast'),
                                           sampling_freq))

In [ ]:
from copy import deepcopy
import resampy
def preproc_set_further(the_set, start_fs, preproc_functions):
    the_set = deepcopy(the_set)
    new_X = []
    for x in the_set.X:
        fs = start_fs
        for preproc_fn in preproc_functions:
            x, fs = preproc_fn(x, fs)
        new_X.append(x)
    the_set.X = new_X
    return the_set

In [ ]:
from braindecode.datautil.signal_target import SignalAndTarget
def create_set(inds, sensor_types):
    X = []
    for i in inds:
        log.info("Load {:s}".format(cleaned_file_names[i]))
        x = load_data(cleaned_file_names[i], preproc_functions, sensor_types=sensor_types)
        X.append(x)
    y = cleaned_labels[inds].astype(np.int64)
    return SignalAndTarget(X, y)

In [ ]:
from autodiag.iterator import ModifiedIterator
from autodiag.batch_modifier import RemoveMinMaxDiff

        
from autodiag.clean import set_jumps_to_zero

In [ ]:
sensor_types = ['EEG', 'EKG']

train_set = create_set(train_inds, sensor_types)
valid_set = create_set(valid_inds, sensor_types)
test_set = create_set(test_inds, sensor_types)

In [ ]:
new_preproc_fn = []#lambda data, fs: (remove_jumps(data,200,400,True,True), fs)]
start_fs = 100
#new_preproc_fn.append(lambda data, fs : (np.clip(data,-200,200),fs))
new_train_set = preproc_set_further(train_set, start_fs, new_preproc_fn)
new_valid_set = preproc_set_further(valid_set, start_fs, new_preproc_fn)
new_test_set = preproc_set_further(test_set, start_fs, new_preproc_fn)

In [ ]:
import numpy as np
from torch import nn
from torch.nn import init

from braindecode.torch_ext.modules import Expression
from braindecode.torch_ext.functions import safe_log, square
from braindecode.torch_ext.util import np_to_var


In [ ]:
from braindecode.torch_ext.util import set_random_seeds
from braindecode.models.shallow_fbcsp import ShallowFBCSPNet
from autodiag.threepathnet import create_multi_start_path_net
cuda = True

set_random_seeds(seed=20170629, cuda=cuda)
input_time_length = 1200
in_chans = 22
early_bnorm = False
extra_conv_stride = 4
later_kernel_len = 5
mean_across_features = True
n_classifier_filters = 100
n_start_filters = 10
drop_prob = 0.5
virtual_chan_1x1_conv = True


model = create_multi_start_path_net(in_chans=in_chans,
        virtual_chan_1x1_conv=virtual_chan_1x1_conv, 
        n_start_filters=n_start_filters, early_bnorm=early_bnorm,
        extra_conv_stride=extra_conv_stride, mean_across_features=mean_across_features,
        later_kernel_len=later_kernel_len,
        n_classifier_filters=n_classifier_filters, drop_prob=drop_prob)



test_input = np_to_var(
    np.ones((2, in_chans, input_time_length, 1), dtype=np.float32))
model(test_input).size()

In [ ]:
from torch import optim
optimizer = optim.Adam(model.parameters())#(model.parameters(), grad_clip_std_factor=2)#optim.Adam(model.parameters())
log.info("Model:\n{:s}".format(str(model)))
if cuda:
    model.cuda()
n_preds_per_input = input_time_length // 2

In [ ]:
import math
class CosineWithWarmRestarts(object):
    def __init__(self, optimizer, batch_period, base_lr, base_wd = 0, update_wd = 0):

        self.optimizer = optimizer
        self.base_lr = base_lr
        self.base_wd = base_wd
        self.batch_iteration = 0
        self.batch_period = batch_period
        self.update_wd = update_wd
        self.m_mult = 1.0
        self.t_mult = 1.0

    def step(self, batch_iteration=None):
        if batch_iteration is None:
            self.batch_iteration = self.batch_iteration = self.batch_iteration + 1
        else:
            self.batch_iteration = batch_iteration
        tt = self.batch_iteration / self.batch_period
        multiplier = 0.5 * (1.0 + math.cos(tt * math.pi))
        cur_lr = self.base_lr * multiplier
        cur_wd = self.base_wd * multiplier
        for i, param_group in enumerate(self.optimizer.param_groups):
            param_group['lr'] = cur_lr
            if (self.update_wd == 1):    param_group['weight_decay'] = cur_wd
        return cur_lr

In [ ]:
class ScheduledOptimizer(object):
    def __init__(self, scheduler):
        assert hasattr(scheduler, 'optimizer')
        self.scheduler = scheduler
    def state_dict(self):
        self.scheduler.optimizer.state_dict()
        
    def step(self):
        self.scheduler.optimizer.step()
        self.scheduler.step()
    def zero_grad(self):
        self.scheduler.optimizer.zero_grad()

In [ ]:

batch_size = 32
log.info("{:d} predictions per input/trial".format(n_preds_per_input))
iterator = CropsFromTrialsIterator(batch_size=batch_size,
                                   input_time_length=input_time_length,
                                  n_preds_per_input=n_preds_per_input)

init_lr = 0.1

n_batches = sum([1 for _ in iterator.get_batches(train_set, shuffle=False)])
max_epochs = 20
momentum = 0.9

In [ ]:
import torch.nn.functional as F

from autodiag.monitors import CroppedDiagnosisMonitor
from braindecode.experiments.experiment import Experiment
from braindecode.datautil.iterators import CropsFromTrialsIterator
from braindecode.experiments.monitors import (RuntimeMonitor, LossMonitor,
                                              CroppedTrialMisclassMonitor,
                                              MisclassMonitor)
from braindecode.experiments.stopcriteria import MaxEpochs

optimizer = optim.SGD(model.parameters(), momentum=momentum, lr=init_lr)
loss_function = lambda preds, targets: F.nll_loss(th.mean(preds, dim=2, keepdim=False),
                                                  targets)
model_constraint = None
monitors = [LossMonitor(), MisclassMonitor(col_suffix='sample_misclass'),
            CroppedDiagnosisMonitor(input_time_length, n_preds_per_input),
            RuntimeMonitor(),]
stop_criterion = MaxEpochs(20)
optimizer = ScheduledOptimizer(CosineWithWarmRestarts(optimizer,batch_period=max_epochs*n_batches,
                                                      base_lr=init_lr))
batch_modifier=None
exp = Experiment(model, new_train_set, new_valid_set, new_test_set, iterator,
                 loss_function, optimizer, model_constraint,
                 monitors, stop_criterion,
                 remember_best_column='valid_misclass',
                 run_after_early_stop=False, batch_modifier=batch_modifier, cuda=cuda)

In [ ]:
(100 * 100 * 60 * 20 * 2) / (1024 ** 2.0)

In [ ]:
exp.setup_training()
exp.run_until_early_stop()

In [ ]:
from autodiag.monitors import compute_preds_per_trial
from braindecode.torch_ext.util import var_to_np
for setname in ('train', 'valid', 'test'):
    dataset = exp.datasets[setname]
    preds_per_batch = [var_to_np(exp.model(np_to_var(b[0]).cuda()))
              for b in exp.iterator.get_batches(dataset, shuffle=False)]
    preds_per_trial = compute_preds_per_trial(
        preds_per_batch, dataset,
        input_time_length=exp.iterator.input_time_length,
        n_stride=exp.iterator.n_preds_per_input)
    mean_preds_per_trial = [np.mean(preds, axis=(0, 2)) for preds in
                                preds_per_trial]
    mean_preds_per_trial = np.array(mean_preds_per_trial)

In [ ]:
np.load('data/models/pytorch/auto-diag/dummy-save-pred/7/test_trial_preds.npy')

In [ ]:
plt.plot(exp.epochs_df.train_loss)

In [ ]:
plt.plot(exp.epochs_df.train_loss)

In [ ]:
plt.plot(exp.epochs_df.train_loss)

In [ ]:
plt.plot(exp.epochs_df.train_loss)